# QuotationTool
In this notebook, we will use the *QuotationTool* to extract quotes from a list of texts. In addition to extracting the quotes, the tool also provides information about who the speakers are, the location of the quotes (and the speakers) within the text, the identified named entities, etc., which can be useful for your text analysis.  

**Note:** This code has been adapted (with permission) from the [GenderGapTracker GitHub page](https://github.com/sfu-discourse-lab/GenderGapTracker/tree/master/NLP/main) and modified to run on a Jupyter Notebook. The quotation tool’s accuracy rate is evaluated in [this article](https://doi.org/10.1371/journal.pone.0245533).

## 1. Setup
Before we begin, we need to import the QuotationTool and initiate it to run in this notebook.

In [ ]:
# import the QuotationTool
from extract_display_quotes import QuotationTool

# initialize the QuotationTool
qt = QuotationTool()

## 2. Load the data
This notebook will allow you to extract quotes directly from a text file (or a number of text files). Alternatively, you can also extract quotes from a text column inside your excel spreadsheet.  

To extract quotes directly from text files, please upload your text files (.txt) below. However, if you already have your texts stored inside an Excel spreadsheet, you can select the second tab to upload your spreadsheet.

In [ ]:
# widget to upload .txt files
tab = qt.upload_files()
tab

Once your files are uploaded, you can run the below code to see a preview of the text in a table format (pandas dataframe).

In [ ]:
# display a preview of the pandas dataframe
qt.text_df.head()

## 3. Extract the quotes
Once your texts have been stored in a pandas dataframe, you can begin to extract the quotes from the texts. You can also extract named entities from your text by setting the named entities you wish to include in the below *inc_ent* variable.

In [ ]:
inc_ent = ['ORG','PERSON','GPE','NORP','FAC','LOC']

quotes_df = qt.get_quotes(inc_ent, create_tree=False)
quotes_df.head()

In general, the quotes are extracted either based on syntactic rules or heuristic rules. Some quotes can be stand-alone in a sentence, or followed by another quote (floating quote) in the same sentence.   

**Quotation symbols:** *Q (Quotation mark), S (Speaker), V (Verb), C (Content)*  

**Named Entities:**  *PERSON (People, including fictional), NORP (Nationalities or religious or political groups), FAC (Buildings, airports, highways, bridges, etc.), ORG (Companies, agencies, institutions, etc.), GPE (Countries, cities, states), LOC (Non-GPE locations, mountain ranges, bodies of water)*

## 4. Display the quotes
Once you have extracted the quotes, you can see a preview of the quotes using spaCy's visualisation tool, displaCy. All you need to do is run the below code and select the text_id you wish to analyse and what entities to show. 

Click the ***Preview*** button to display the quotes, and click ***Save Preview*** to save them as an html file. Lastly, you also have the option to show the top five Named Entities mentioned in the speakers and quotes by clicking the ***Top 5 Entitites*** button.

In [ ]:
box = qt.analyse_quotes(inc_ent)
box

## 5. Save your quotes
Finally, you can save the quotes pandas dataframe into an Excel spreadsheet and download them to your local computer.

In [ ]:
# save quotes_df into an Excel spreadsheet
quotes_df.to_excel('./output/quotes.xlsx', index=False)